In [1]:
cd desktop\\Data analysis\\Datasets

C:\Users\Sagar\desktop\Data analysis\Datasets


In [2]:
df = pd.read_csv('Social_Network_Ads.csv')
df.head()

<IPython.core.display.Javascript object>

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [3]:
X = np.array(df.iloc[:,2:4])
y = np.array(df.iloc[:,4:])

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=0)

Standardization

In [5]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Applying ML algorithm RF

In [8]:
from sklearn.ensemble import RandomForestClassifier

In [9]:
classifier = RandomForestClassifier(n_estimators=10,criterion='entropy',random_state=50)
classifier.fit(X_train,y_train)

C:\Users\Sagar\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=50, verbose=0,
                       warm_start=False)

In [10]:
y_pred = classifier.predict(X_test)

In [11]:
from sklearn.metrics import confusion_matrix
cm  = confusion_matrix(y_test,y_pred)

In [12]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test,y_pred)
accuracy

0.92

Now, we will work with RandomizedSearchCV to find the best parameters

In [13]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [14]:
est = RandomForestClassifier(n_jobs=-1)
rf_p_dist={'max_depth':[3,5,10,None],
              'n_estimators':[10,100,200,300,400,500],
              'max_features':randint(1,3),
               'criterion':['gini','entropy'],
               'bootstrap':[True,False],
               'min_samples_leaf':randint(1,4),
              }

In [67]:
#Use this function if you wanna execute via function otherwise use the variable
# def hypertuning_rscv(est, p_distr, nbr_iter,X,y):
#     rdmsearch = RandomizedSearchCV(est, param_distributions=p_distr,
#                                   n_jobs=-1, n_iter=nbr_iter, cv=9)
#     #CV = Cross-Validation ( here using Stratified KFold CV)
#     rdmsearch.fit(X,y)
#     ht_params = rdmsearch.best_params_
#     ht_score = rdmsearch.best_score_
#     return ht_params, ht_score


random_cv = RandomizedSearchCV(estimator=est,
                              param_distributions=rf_p_dist,
                              n_jobs=-1,n_iter=40,
                              cv=5,scoring='neg_mean_absolute_error',verbose=5,
                              return_train_score=True,random_state=40)

In [68]:
# rf_parameters, rf_ht_score = hypertuning_rscv(est, rf_p_dist, 40, X, y)

#To find execution time;here, we use variable random_cv
from datetime import datetime #Only if we want to find the time
# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_cv.fit(X_train,y_train)
timer(start_time) # timing ends here for "start_time" variable


Fitting 5 folds for each of 40 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   11.3s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:   41.8s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:  1.7min finished
C:\Users\Sagar\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\Sagar\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Sagar\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarn


 Time taken: 0 hours 1 minutes and 41.01 seconds.


In [17]:
# print(rf_parameters)
# print(rf_ht_score)

print(random_cv.best_params_)
print(random_cv.best_score_)

{'bootstrap': True, 'criterion': 'entropy', 'max_depth': 3, 'max_features': 2, 'min_samples_leaf': 1, 'n_estimators': 400}
0.9125


Now, we are going to use the best parameters found by RandomizedSearchCV in model building...

In [18]:
classifier = RandomForestClassifier(bootstrap=True,criterion='entropy',max_depth=3,max_features=2,min_samples_leaf=1,
                                   n_estimators=400,random_state=10)
classifier.fit(X_train,y_train)

C:\Users\Sagar\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=3, max_features=2, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=400,
                       n_jobs=None, oob_score=False, random_state=10, verbose=0,
                       warm_start=False)

In [19]:
y_pred = classifier.predict(X_test)

In [20]:
cm = confusion_matrix(y_test,y_pred)

In [21]:
accuracy = accuracy_score(y_test,y_pred)
accuracy

0.94

We can also check with cross_val_score

In [22]:
from sklearn.model_selection import cross_val_score

In [23]:
score = cross_val_score(classifier,X,y,cv=10,scoring='accuracy').mean()

C:\Users\Sagar\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Sagar\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Sagar\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Sagar\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y wa

In [24]:
score

0.8997701688555347

Let's now play with Xgboost Classifier

Timer:

In [50]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [6]:
from xgboost import XGBClassifier

In [15]:
clf = XGBClassifier(random_state=40)
clf.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=40,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [16]:
y_pred = clf.predict(X_test)#max_depth, learning_rate, n_estimators,booster, n_jobs,gamma,ramdom_state

In [17]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)

In [18]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test,y_pred)
accuracy

0.94

Let's play with hyperparameter tuning for Xgboost

In [14]:
from sklearn.model_selection import RandomizedSearchCV )
#max_depth, learning_rate, n_estimators,booster, n_jobs,gamma,ramdom_state,min_child_weight,base_score

In [49]:
est = XGBClassifier(n_jobs=-1)
rf_p_dist={'max_depth':[3,5,10,13,15],
              'n_estimators':[10,100,200,300,400,500,600,700,800,900,1000],
               'booster':['gbtree','gblinear','dart'],
               'learning_rate':[0.05,0.1,0.15,0.2],
               'base_score':[0.25,0.50,0.75,0.99],
               'min_child_weight':[1,2,3,4,5],
               'gamma':[0.0,0.1,0.2,0.3,0.4]
              }

In [31]:
# Use this function if you wanna execute via function otherwise use the variable
# def hypertuning_rscv(est, p_distr, nbr_iter,X,y):
#     rdmsearch = RandomizedSearchCV(est, param_distributions=p_distr,
#                                   n_jobs=-1, n_iter=nbr_iter, cv=5)
#     #CV = Cross-Validation ( here using Stratified KFold CV)
#     rdmsearch.fit(X,y)
#     ht_params = rdmsearch.best_params_
#     ht_score = rdmsearch.best_score_
#     return ht_params, ht_score



random_cv = RandomizedSearchCV(estimator=est,
                              param_distributions=rf_p_dist,
                              n_jobs=-1,n_iter=40,
                              cv=5,scoring='neg_mean_absolute_error',verbose=5,
                              return_train_score=True,random_state=40)

In [58]:
#rf_parameters, rf_ht_score = hypertuning_rscv(est, rf_p_dist, 5, X, y)

#To find execution time;here, we use variable random_cv
from datetime import datetime #Only if we want to find the time
# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_cv.fit(X_train,y_train)
timer(start_time) # timing ends here for "start_time" variable

Fitting 5 folds for each of 40 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   24.3s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:   55.5s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:  1.9min finished
C:\Users\Sagar\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\Sagar\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Sagar\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarn


 Time taken: 0 hours 1 minutes and 56.35 seconds.


In [60]:
# print(rf_parameters)
# print(rf_ht_score)
print(random_cv.best_params_)
print(random_cv.best_score_)

{'n_estimators': 700, 'min_child_weight': 2, 'max_depth': 10, 'learning_rate': 0.15, 'gamma': 0.4, 'booster': 'gbtree', 'base_score': 0.99}
-0.09666666666666666


In [61]:
clf = XGBClassifier(n_estimators=700, min_child_weight=2, max_depth=10, learning_rate=0.15, gamma=0.4, booster = 'gbtree', base_score = 0.99,cv=5)
clf.fit(X_train,y_train)

XGBClassifier(base_score=0.99, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, cv=5, gamma=0.4,
              learning_rate=0.15, max_delta_step=0, max_depth=10,
              min_child_weight=2, missing=None, n_estimators=700, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [62]:
y_pred = clf.predict(X_test)

In [63]:
cm = confusion_matrix(y_test,y_pred)

In [64]:
accuracy = accuracy_score(y_test,y_pred)
accuracy

0.94

In [65]:
from sklearn.model_selection import cross_val_score
score = cross_val_score(clf,X,y,cv=5)


C:\Users\Sagar\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Sagar\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [66]:
score.mean()

0.8648253633380214